In [123]:
import numpy as np
import pandas as pd

In [124]:
lr = 2
epoch = 300
classnum = 10
img_h = 28
img_w = 28
batch_size = 128

In [125]:
def get_image_sector(image, str_from, str_to, col_from, col_to):
    sector = image[:, str_from:str_to, col_from:col_to]
    return sector.reshape(-1, 1, str_to-str_from, col_to-col_from)

In [126]:
def conv2d(image, kernel):
    for i in range(int(len(image)/batch_size)):
        batch_start, batch_end = ((i*batch_size), (i+1)*batch_size)
        layer = image[batch_start:batch_end]
        layer = layer.reshape(layer.shape[0], img_h, img_w)
        sects = []
        for str_from in range(layer.shape[1]-kernel.shape[1]):
            for col_from in range(layer.shape[2]-kernel.shape[2]):
                sect = get_image_sector(layer, str_from, str_from+kernel.shape[1], col_from, col_from+kernel.shape[2])
                sects.append(sect)
        expanded_input = np.concatenate(sects, axis=1)
    return expanded_input.reshape(np.prod(expanded_input.shape), -1).dot(kernel)

In [127]:
def pool(image, kernel):
    for i in range(int(len(image)/batch_size)):
        batch_start, batch_end = ((i*batch_size), (i+1)*batch_size)
        layer = image[batch_start:batch_end]
        layer = layer.reshape(layer.shape[0], img_h, img_w)
        sects = []
        for str_from in range(layer.shape[1]-kernel[0]):
            for col_from in range(layer.shape[2]-kernel[1]):
                sect = get_image_sector(layer, str_from, str_from+kernel[0], col_from, col_from+kernel[1])
                sects.append(sect)
    return sects

In [128]:
def sigm2deriv(x):
    return sigmoid(x)*(1-sigmoid(x))

In [133]:
def sigmoid(x):
    return 1+(1+e**(-x))

In [134]:
def softmax(x):
    temp = np.exp(x)
    return temp/np.sum(temp, axis=1, keepdims=True)

In [135]:
def Flatten(x):
    return x.reshape(x[0]*x[1],-1)

In [137]:
class Layer:
    weigths = 0
    def __init__(self, with_w=False, derivative=0, function = 0):
        self.act = function
        self.deriv = derivative
        self.flag = with_w
   
    def init_weigths(self ,size, shape):
        self.weigths = np.random.uniform(-1,1, size)
        self.weigths = self.weigths.reshape(shape)
        
    def dropout(layer):
        mask = np.random.randint(2, size=layer.shape)
        return layer*mask*2

    def forward(self, input):
        pass
    
    def backward():

In [138]:
class Linear(Layer):
    inp = 0
    out = 0
    def __init__(self, in_features, out_features, bias = False):
        super(Linear, self).__init__()
        self.inp = in_features
        self.out = out_features
        self.init_weigths(self.inp**2*self.out**2, [[self.inp, self.inp], [self.out, self.out]])
        self.forward = matmul
    
    def matmul(self, input):
        assert([input.shape[2], input.shape[3]]==self.weigths.shape[1])
        input = input.reshape([input.shape[0], input.shape[1]],[input.shape[2], input.shape[3]])
        return np.matmul(input, self.weigths)
      

In [139]:
class Conv2d(Layer):
    in_channels = 0
    out_channels = 0
    kernel_size = 0
    padding = 0
    def __init__(self, in_chan, out_chan, kernel_size, padding=False):
        self.in_channels = in_chan
        self.out_channels = out_chan
        self.kernel_size = kernel_size
        self.padding = padding
        self.init_weigths(np.prod(kernel_size), kernel_size)
        self.forward = conv
    
    def conv(self, input):
        return conv2d(input, self.weigths)

In [140]:
class Pool2d(Layer):
    kernel_size = 0
    mode = 0
    def __init__(self, kernel_size, mode="Avg"):
        self.kernel_size = kernel_size
        self.mode = mode
    
    def forward(self, input):
        if self.mode =="Max":
            sects = pool(input, self.kernel_size)
            out = np.zeros(len(sects))
            for i in range(len(sects)):
                out[i] = sects[i].argmax(1)
            return out
        elif self.mode =="Avg":
            sects = pool(input, self.kernel_size)
            out = np.zeros(len(sects))
            for i in range(len(sects)):
                out[i] = np.sum(sects[i])/np.prod(self.kernel_size)
            return out

In [136]:
def CrossEntropyLoss(predict, labels):
    assert(predict.shape[0]==len(labels))
    predict = predict.argmax(1)
    loss = 0
    for i in range(len(labels)):
        loss += labels[i]*np.log2(predict[i])
    loss *= -1
    return loss

In [141]:
data = pd.read_csv("mnist_train.csv")
train_label, train_data = data["label"], data.drop(["label"], axis=1)
train_data = train_data.to_numpy()
train_label = train_label.to_numpy()